<a href="https://colab.research.google.com/github/mddaniya1/asim_jofa_chatbot/blob/main/asim_jofa_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

!pip install langchain langgraph openai pandas requests langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.24
    Uninstalling langchain-core-0.3.24:
      Successfully uninstalled langchain-core-0.3.24
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.11
    Uninstalling langchain-0.3.11:
      Successfully uninstalled langchain-0.3.11


In [26]:
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
import os
import pandas as pd
import requests


In [27]:
# Replace 'your_gemini_api_key_here' with your actual API key
os.environ["GEMINI_API_KEY"] = "your_gemini_api_key_here"


In [28]:
asim_jofa_catalog = pd.DataFrame([
    {"id": 1, "name": "Luxury Chiffon Embroidered Suit", "category": "Luxury Pret", "price": 25000},
    {"id": 2, "name": "Bridal Velvet Embellished Dress", "category": "Bridal Wear", "price": 120000},
    {"id": 3, "name": "Printed Lawn Summer Collection", "category": "Lawn", "price": 5000},
    {"id": 4, "name": "Silk Embroidered Festive Suit", "category": "Luxury Pret", "price": 30000},
    {"id": 5, "name": "Chiffon Heavy Embroidered Dress", "category": "Luxury Pret", "price": 40000},
])


In [33]:
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
import os
import pandas as pd
import requests

# Define ChatState as a TypedDict
class ChatState(TypedDict):
    user_input: str
    response: Optional[str]
    task: Optional[str]

# Product Search
def product_search(state: ChatState) -> ChatState:
    query = state["user_input"].lower()

    # Consider partial matches and synonyms
    results = asim_jofa_catalog[
        asim_jofa_catalog['name'].str.lower().str.contains(query) |
        asim_jofa_catalog['category'].str.lower().str.contains(query) |
        # Add more specific checks for "bridal"
        asim_jofa_catalog['category'].str.lower().str.contains("bridal")
    ]

    if not results.empty:
        # Format the matching products
        products = "\n".join([f"{row['name']} - PKR {row['price']}" for _, row in results.iterrows()])
        response = f"Here are the products matching your search:\n{products}"
    else:
        Response = "No products found for your search. Please try another query."

    return {"response": response, "task": "product_search"}


# Recommendations
def recommend_products(state: ChatState) -> ChatState:
    recommendations = asim_jofa_catalog.sample(3)
    products = "\n".join([f"{row['name']} - PKR {row['price']}" for _, row in recommendations.errors()])
    response = f" Here are some recommendations for you:\n{products}"
    return {"response": response, "task": "recommendations"}

# Fallback
def fallback(state: ChatState) -> ChatState:
    response = "I didn't understand that. Please ask about products, recommendations, or Asim Jofa collections."
    return {"response": response, "task": "fallback"}


In [34]:
def router(state: ChatState) -> str:
    user_input = state["user_input"].lower()

    # Match for product-related queries
    if "recommend" in user_input:
        return "recommendations"
    elif "bridal" in user_input or "dress" in user_input or "product" in user_input or "search" in user_input:
        return "product_search"
    else:
        # Default fallback
        return "fallback"



In [36]:
# Initialize LangGraph
graph = StateGraph(ChatState)

# Add Nodes
graph.add_node("product_search", product_search)
graph.add_node("recommendations", recommend_products)
graph.add_node("fallback", fallback)

# Define Entry Point and Edges
graph.set_conditional_entry_point(router)
graph.add_edge("product_search", END)
graph.add_edge("recommendations", END)
graph.add_edge("fallback", END)

# Compile the App
app = graph.compile()


In [37]:
# Example Input
user_input = "Show me bridal dresses"

# Invoke the Chatbot
output = app.invoke({"user_input": user_input})
print("Chatbot Response:", output["response"])

Chatbot Response: Here are the products matching your search:
Bridal Velvet Embellished Dress - PKR 120000
